In [11]:
import os
import cv2
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.utils import *

In [12]:
images = os.listdir(r"C:\Users\himu1\OneDrive\Desktop\dataset\CoMoFoD_small_v2")
print("Total image found: {}".format(len(images)))

Total image found: 10402


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Data Preprocessing
# Load the dataset (Assuming you have already downloaded and extracted the dataset)
# Adjust the file paths according to your dataset directory structure
image_dir = r'C:\Users\himu1\OneDrive\Desktop\dataset\original'
colored_mask_dir = r'C:\Users\himu1\OneDrive\Desktop\dataset\colored_mask'
binary_mask_dir = r'C:\Users\himu1\OneDrive\Desktop\dataset\binary_mask'
forged_image_dir = r'C:\Users\himu1\OneDrive\Desktop\dataset\forged_image'

# Define a function to load and preprocess a single image set
def load_image_set(image_path, colored_mask_path, binary_mask_path, forged_image_path):
    # Load and resize the images
    image = keras.preprocessing.image.load_img(image_path, target_size=(512, 512))
    colored_mask = keras.preprocessing.image.load_img(colored_mask_path, target_size=(512, 512))
    binary_mask = keras.preprocessing.image.load_img(binary_mask_path, target_size=(512, 512))
    forged_image = keras.preprocessing.image.load_img(forged_image_path, target_size=(512, 512))

    # Convert the images to arrays and normalize pixel values
    image = keras.preprocessing.image.img_to_array(image) / 255.0
    colored_mask = keras.preprocessing.image.img_to_array(colored_mask) / 255.0
    binary_mask = keras.preprocessing.image.img_to_array(binary_mask) / 255.0
    forged_image = keras.preprocessing.image.img_to_array(forged_image) / 255.0

    return image, colored_mask, binary_mask, forged_image

# Collect all image set paths
image_set_paths = []
for i in range(1, 201):  # Assuming you have 260 image sets in the dataset
    image_path = f'{image_dir}/{i}_O.png'
    colored_mask_path = f'{colored_mask_dir}/{i}_M.png'
    binary_mask_path = f'{binary_mask_dir}/{i}_B.png'
    forged_image_path = f'{forged_image_dir}/{i}_F.png'
    image_set_paths.append((image_path, colored_mask_path, binary_mask_path, forged_image_path))

# Load and preprocess all image sets
images = []
colored_masks = []
binary_masks = []
forged_images = []
for paths in image_set_paths:
    image, colored_mask, binary_mask, forged_image = load_image_set(*paths)
    images.append(image)
    colored_masks.append(colored_mask)
    binary_masks.append(binary_mask)
    forged_images.append(forged_image)

# Convert the lists to arrays
images = np.array(images)
colored_masks = np.array(colored_masks)
binary_masks = np.array(binary_masks)
forged_images = np.array(forged_images)

# Split the dataset into training and testing sets
images_train, images_test, masks_train, masks_test = train_test_split(images, colored_masks, test_size=0.2, random_state=42)

In [14]:
class PreprocessingBlock(keras.Model):
    def __init__(self):
        super(PreprocessingBlock, self).__init__()
        self.resize = layers.experimental.preprocessing.Resizing(256, 256, 3)
        self.conv = layers.Conv2D(64, kernel_size=(7, 7), strides=2, padding='same')
        self.batch_norm = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.max_pool = layers.MaxPooling2D(pool_size=(2, 2), strides=2)

    def call(self, inputs):
        x = self.resize(inputs)
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.max_pool(x)
        return x

In [15]:
import tensorflow as tf
import numpy as np

class TransformerBlock(layers.Layer):
    def __init__(self, d_model, num_heads, projection_dim, dff):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation="relu"),
            layers.Dense(d_model),
        ])
        self.layer_norm_1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm_2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout_1 = layers.Dropout(0.1)
        self.dropout_2 = layers.Dropout(0.1)

    def call(self, inputs, training):
        attention_output = self.attention(inputs, inputs)
        attention_output = self.dropout_1(attention_output, training=training)
        attention_output = self.layer_norm_1(inputs + attention_output)
        ffn_output = self.ffn(attention_output)
        ffn_output = self.dropout_2(ffn_output, training=training)
        return self.layer_norm_2(attention_output + ffn_output)

def positional_encoding_2D(row, col, d_model):
    position_row = np.arange(row)[:, np.newaxis]
    position_col = np.arange(col)[np.newaxis, :]
    angle_row = position_row / np.power(10000, (2 * (position_row // 2)) / np.float32(d_model))
    angle_col = position_col / np.power(10000, (2 * (position_col // 2)) / np.float32(d_model))

    # Apply sine to even indices in the array
    angle_row[:, 0::2] = np.sin(angle_row[:, 0::2])
    angle_col[:, 0::2] = np.sin(angle_col[:, 0::2])

    # Apply cosine to odd indices in the array
    angle_row[:, 1::2] = np.cos(angle_row[:, 1::2])
    angle_col[:, 1::2] = np.cos(angle_col[:, 1::2])

    # Reshape and repeat to match the dimensions
    angle_row = angle_row[np.newaxis, :, np.newaxis, :]
    angle_col = angle_col[np.newaxis, :, :, np.newaxis]
    angle_row = tf.repeat(angle_row, repeats=col, axis=2)
    angle_col = tf.repeat(angle_col, repeats=row, axis=1)

    # Combine row and column angles
    pos_encoding = tf.concat([angle_row, angle_col], axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)



# Transformer branch
class TransformerBranch(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, projection_dim, dff, input_shape):
        super(TransformerBranch, self).__init__()
        self.num_layers = num_layers
        self.d_model = d_model
        self.num_heads = num_heads
        self.projection_dim = projection_dim
        self.dff = dff
        self.input_shape_custom = input_shape  # Choose a different name for the attribute

        self.embedding = layers.Dense(self.d_model)
        self.pos_encoding = positional_encoding_2D(self.input_shape_custom[0], self.input_shape_custom[1], self.d_model)

        self.transformer_blocks = [TransformerBlock(self.d_model, self.num_heads, self.projection_dim, self.dff)
                                    for _ in range(self.num_layers)]
        self.dropout = layers.Dropout(0.3)

    def call(self, inputs):
        x = self.embedding(inputs)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :tf.shape(x)[1], :]
        x = self.dropout(x)

        for i in range(self.num_layers):
            x = self.transformer_blocks[i](x)

        return x

    def point_wise_feed_forward_network(self, embedding_dim, ff_dim):
        return tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embedding_dim)
        ])

#     def call(self, inputs):
#         x = self.layer_normalization1(inputs)
#         attention_output = self.mhsa_block(x)
#         attention_output = self.dropout1(attention_output)
#         x1 = tf.add(inputs, attention_output)
#         x = self.layer_normalization2(x1)
#         ffn_output = self.ffn_block(x)
#         ffn_output = self.dropout2(ffn_output)
#         x2 = tf.add(x1, ffn_output)
#         return x2



class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, d_model, num_heads, projection_dim):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.projection_dim = projection_dim
        self.d_model = d_model

        assert self.projection_dim % self.num_heads == 0, "Projection dimension must be divisible by the number of heads."
        self.depth = self.projection_dim // self.num_heads

        self.query_dense = layers.Dense(self.projection_dim)
        self.key_dense = layers.Dense(self.projection_dim)
        self.value_dense = layers.Dense(self.projection_dim)
        self.combine_heads = layers.Dense(self.d_model)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        scaled_score = score / tf.math.sqrt(tf.cast(tf.shape(key)[-1], tf.float32))
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention_logits = tf.matmul(query, key, transpose_b=True)
        attention_logits = attention_logits / tf.math.sqrt(tf.cast(self.depth, tf.float32))
        attention_weights = tf.nn.softmax(attention_logits, axis=-1)
        attention_outputs = tf.matmul(attention_weights, value)
        attention_outputs = tf.transpose(attention_outputs, perm=[0, 2, 1, 3])
        attention_outputs = tf.reshape(attention_outputs, (batch_size, -1, self.projection_dim))
        attention_outputs = self.combine_heads(attention_outputs)
        return attention_outputs

# Define the MLP layer
class MLPLayer(keras.layers.Layer):
    def __init__(self, d_model):
        super(MLPLayer, self).__init__()
        self.fc1 = layers.Dense(d_model * 4, activation='relu')
        self.fc2 = layers.Dense(d_model)
        self.dropout = layers.Dropout(0.1)

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


In [16]:
class CNNBranch(keras.Model):
    def __init__(self):
        super(CNNBranch, self).__init__()
        self.conv_block1 = self._build_conv_block(7, 256, 64, 64)
        self.conv_block2 = self._build_conv_block(8, 512, 32, 32)
        self.conv_block3 = self._build_conv_block(6, 1024, 16, 16)

    def _build_bottleneck(self, filters):
        bottleneck = keras.Sequential([
            layers.Conv2D(filters, kernel_size=(1, 1), strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(filters, kernel_size=(3, 3), strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(filters, kernel_size=(1, 1), strides=1, padding='same'),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Add()
        ])
        return bottleneck

    def _build_conv_block(self, num_bottlenecks, filters, height, width):
        conv_block = keras.Sequential()
        for _ in range(num_bottlenecks):
            bottleneck = self._build_bottleneck(filters)
            conv_block.add(bottleneck)
        conv_block.add(layers.Conv2D(filters, kernel_size=(1, 1), strides=1, padding='same'))
        conv_block.add(layers.BatchNormalization())
        conv_block.add(layers.ReLU())
        conv_block.add(layers.MaxPooling2D(pool_size=(2, 2)))
        conv_block.build(input_shape=(None, height, width, filters))
        return conv_block

    def call(self, inputs):
        x = self.conv_block1(inputs)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        return x

In [17]:
class FeatureCouplingDownsampling(keras.Model):
    def __init__(self):
        super(FeatureCouplingDownsampling, self).__init__()
        self.channel_align = layers.Conv2D(64, kernel_size=(1, 1), strides=1, padding='same')
        self.avg_pool = layers.AveragePooling2D(pool_size=(4, 4), strides=4)
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
        self.gelu = layers.Activation('gelu')

    def call(self, inputs):
        x = self.channel_align(inputs)
        x = self.avg_pool(x)
        x = self.layer_norm(x)
        x = self.gelu(x)
        return x


class FeatureCouplingUpsampling(keras.Model):
    def __init__(self):
        super(FeatureCouplingUpsampling, self).__init__()
        self.localization_reshape = layers.Reshape(target_shape=(768, 16, 16))
        self.channel_align = layers.Conv2D(256, kernel_size=(1, 1), strides=1, padding='same')
        self.batch_norm = layers.BatchNormalization(epsilon=1e-6)
        self.relu = layers.ReLU()
        self.up_sampling = layers.UpSampling2D(size=(4, 4))

    def call(self, inputs):
        x = self.localization_reshape(inputs)
        x = self.channel_align(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.up_sampling(x)
        return x

In [18]:
class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv_block1 = self._build_conv_block(64, kernel_size=5, strides=2)
        self.conv_block2 = self._build_conv_block(128, kernel_size=5, strides=2)
        self.conv_block3 = self._build_conv_block(256, kernel_size=5, strides=2)
        self.conv_block4 = self._build_conv_block(512, kernel_size=5, strides=1)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(1)
        self.activation = layers.Activation('sigmoid')

    def _build_conv_block(self, filters, kernel_size, strides):
        conv_block = keras.Sequential([
            layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same'),
            layers.BatchNormalization(),
            layers.LeakyReLU(0.2)
        ])
        return conv_block

    def call(self, inputs):
        x = self.conv_block1(inputs)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.activation(x)
        return x

In [20]:
# Instantiate the CNN-T-GAN model
embedding_dim = 64
num_heads = 8
ff_dim = 1024
dropout_rate = 0.1
num_layers = 4
d_model = 256
projection_dim = 64
dff = 1024
cnn_branch = CNNBranch()

# Instantiate the pre-processing block
preprocessing_block = PreprocessingBlock()

# Define the inputs for the model
inputs = keras.Input(shape=(512, 512, 3))  # Adjust the input shape based on your images

# Pass the inputs through the pre-processing block
preprocessed_output = preprocessing_block(inputs)

# Get the input shape from the preprocessed_output tensor
input_shape = preprocessed_output.shape[1:]

# Pass the preprocessed output through the transformer branch
transformer_branch = TransformerBranch(num_layers, d_model, num_heads, projection_dim, dff, input_shape)
transformer_output = transformer_branch(preprocessed_output)
cnn_output = cnn_branch(preprocessed_output)

# Resize the transformer_output
transformer_output_resized = tf.image.resize(
    transformer_output, (64, 64), method=tf.image.ResizeMethod.BILINEAR
)

# Resize the cnn_output
cnn_output_resized = tf.image.resize(
    cnn_output, (64, 64), method=tf.image.ResizeMethod.BILINEAR
)

# Concatenate the resized cnn_output with transformer_output
combined_output = layers.Concatenate()([cnn_output_resized, transformer_output_resized])

# Define the combined model
model = keras.Model(inputs=inputs, outputs=combined_output)

ValueError: Exception encountered when calling layer "sequential_17" (type Sequential).

A merge layer should be called on a list of inputs. Received: input_shape=(None, 64, 64, 256) (not a list of shapes)

Call arguments received by layer "sequential_17" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 64, 64, 256), dtype=float32)
  • training=False
  • mask=None

In [ ]:
model.summary()

In [ ]:
# Create the final model
model = keras.Model(inputs=inputs, outputs=combined_output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Define the number of epochs and steps per epoch
epochs = 10
batch_size = 32  # Update with your desired batch size
steps_per_epoch = len(images_train) // batch_size

# Train the model
model.fit(images_train, masks_train, batch_size=batch_size, epochs=epochs, validation_data=(images_test, masks_test))

# Evaluate the model on the testing dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)